# ENTITY MATCHING EXPERIMENT

#### Data description

The data provided is in csv format and contains two features
and a label. The unnamed first column has been analyzed and
it contains only unique values with certain values missing from the sequence thereby
establishing the following assumption.

Assumption: The unnamed first column is the index column where the
missing values from the sequence indicate a data cleaning task 
performed prior to handing over the dataset

Note: The above assumption does not rule out the further need for
data cleaning and preprocessing. It is only to take the unnamed column
as index for the dataset.

The following are the features of the dataset:

"entity_1" : This contains company names with extra spaces and string length as low as 1
"entity_2" : This contains company names with extra spaces and string length as low as 1

The label column in the dataset is named "tag" and it is a binary integar data with values 1 and 0

The data set is imbalanced with the negative class accounting to 60% of the examples
and positive class accounting to 40% of the example

There are no missing values in any of the features


#### Data Cleaning process


#### Balancing the dataset


#### Model Selection (Answer to the first part of task 1)

There are various ways in which this task can be handled
as described below:

1. Siamese Networks
2. Attention mechanisms
3. Transformers
4. Pre-trained models like BERT for generating embeddings and vector space

The above list is not exhaustive and hence one can take advantage
of various new research like the one suggested in the paper titled
"Business Entity Matching with Siamese Graph Convolutional Networks"
written by employees and trainees at IBM Zurich. The link for which is given below

https://arxiv.org/abs/2105.03701

This architecture uses BERT + GCN and according to the paper it generates
better results in terms of robustness to semantic meanings and title endings etc

The model selected for this task is a Siamese Network. The Siamese network
is an architecture that is highly popular in tasks where similarity is to be 
measured between pairs of input data.

The key idea is to have two identical subnetworks that share same parameters,
weights and biases a.k.a the siamese twins

This architecture when developed objectivises robustness to variation while learning
meaningful representations to the inputs

The distance metric used here is a L1 norm since it is more robust

The model architecture presented here consist of an embedding layer,
followed by shared LSTM units followed by L1 distance.
 This is followed by fully connected layers
and dropouts for regularization thereby avoiding overfitting.

LSTM is chosen for the shared architecture as it is highly efficient
in capturing long term dependencies and relation ships in data such as company names

The order of words is quite crucial in company names and this is effectively captured by an LSTM unit

Embedding layers capture semantic meanings between words by representing them in
continuous vector space. This can be helpful to capture the subtleties in company names

The final prediction involves usage of a sigmoid since we are performing binary classification

Loss is calculated using binary cross entropy as done for various binary classification problems

Optimizer used: Adam

No Hyperparameter tuning has been performed as it is not required for the given dataset and problem
but if the dataset increases in size or while putting the model to production this may be completely
necessary.

There are a few downsides to this architecture as it is complex and requires heavy computational
resources. It requires large amount of data and tuning it for hyperparameter becomes crucial
despite the computation expense.

It can be difficult to interpret the internal working of model related to how it makes decisions


In [1]:
# import all required libraries and dependencies

# python's batteries included libraries
from pathlib import Path
import re
import string
import math

# for data processing and EDA
import spacy
import scipy
import numpy as np
import pandas as pd

# Model building
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Dropout
from sklearn.feature_extraction.text import CountVectorizer

# viz 
import matplotlib.pyplot as plt

### Data Loading Script

In [2]:
# Function to fetch data from the prescribed path

def load_data(file_name, index_col="Unnamed: 0"):
    """
        There is a prior assumption that the
        dataset is stored under the "data"
        directory in the current working directory
    """

    ## TODO: Add validation to check the Unnamed: 0 col as duplicated or drop
    return pd.read_csv(
        Path().cwd() / "data" / file_name,
        index_col=index_col
    )

df = load_data("ds_challenge_alpas_indexed.csv")

# df.to_csv(Path().cwd() / "data" / "ds_challenge_alpas_indexed.csv", index_label="index_col")

### Basic EDA

In [3]:
df.head()

,index_col,entity_1,entity_2,tag
0,3137667,preciform A.B,Preciform AB,1
1,5515816,degener staplertechnik vertriebs-gmbh,Irshim,0
2,215797,Alltel South CaroliNA Inc,alltel south carolina INC.,1
3,1004621,cse Corporation,Cse Corp,1
4,1698689,Gruppo D Motors Srl,gruppo d motors Sociedad de Resposabilidad Lim...,1


In [9]:
# Class distribution in the labels

df.tag.value_counts() / df.tag.shape[0] * 100

0    59.095513
1    40.904487
Name: tag, dtype: float64

In [5]:
df.shape

(7042846, 4)

In [6]:
df.sort_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042846 entries, 0 to 7042845
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   index_col  int64 
 1   entity_1   object
 2   entity_2   object
 3   tag        int64 
dtypes: int64(2), object(2)
memory usage: 214.9+ MB


### Data Cleaning

The data cleaning process covers the following tasks

1. Removing Punctuations
2. Lower casing the strings
3. Stripping extra spaces
4. Deduplication of records
5. Removing records where both the features have length less than or equal to 3 

(This does not provide any sementic meaning for company names thereby leading to learning unwanted patterns for the model and hence need to be removed. The quantity of such data is considerably low and can be ignored if more robust models like S-GCN architecture are to be constructed)

even after the data cleaning there are no missing values or empty strings

In [5]:
# Remove punctuation and make everything lower case
# re.escape used to put escape char wherever necessary

def preprocess_data(cols):
    df_cols = df.columns

    for col in cols: assert col in df_cols, f"{col} not in dataset column names"

    for col in cols:
        print(f"removing punctuations from {col}")
        df[col] = df[col].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
        print(f"lower casing {col}")
        df[col] = df[col].apply(lambda x: x.lower().strip())

preprocess_data(["entity_1", "entity_2"])


removing punctuations from entity_1
lower casing entity_1
removing punctuations from entity_2
lower casing entity_2


In [6]:
# de duplicate all records
def deduplicated_records(df, strategy=False):

    duplicate_mask = df.duplicated(keep=strategy)

    return df[~duplicate_mask]


df_deduplicated = deduplicated_records(df)
df_deduplicated.tag.value_counts()

0    4162006
1    2880840
Name: tag, dtype: int64

In [15]:
df_deduplicated.isnull().sum()

index_col    0
entity_1     0
entity_2     0
tag          0
dtype: int64

In [22]:
df_deduplicated.shape

(3521423, 3)

In [25]:
# remove short company names

def remove_short_names(df, ln=2):
    mask = (df.entity_1.str.len() > ln) & (df.entity_2.str.len() > ln)
    return df[mask]

df_clean = remove_short_names(df_deduplicated, ln=3)

### Balancing the dataset

The dataset can be easily balanced by using imblearn.over_sampling.SMOTE
The balancing strategy according to the commented logic below is as fellows:

1. We use SMOTE to over sample the dataset thereby increasing its size
2. Further we use RandomUnderSampler to under sample the dataset in order to bring it back to approximately its original size

Performing over sampling is compute expensive
for local runs. Hence Right now we shall
select and balance the dataset manual for trial runs
This introduces a selection bias in the dataset
But nevertheless, we can perform SMOTE or SMOTE-ENN
or SMOTE with RandomUnderSampler to bring back the
dataset size approx to the original.

This can be done with distributed training in Apache beam
A sample pipeline for which has been developed along with this
Submission.

Caution: The Apache Beam pipeline is highly error prone and requires debugging

In [ ]:
# Logic to balance the dataset using
# Synthetic Minority Oversampling Technique
# SMOTE-ENN can also be used here with embeddings

# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import SMOTE
# from sklearn.feature_extraction.text import TfidfVectorizer

# X_train = X_train.entity_1 + "|" + X_train.entity_2

# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# oversample = SMOTE(random_state=42)
# X_bal, y_bal = oversample.fit_resample(X_train_tfidf, y_train)

# X_bal = X_bal.str.split("|", expand=True)

# # Rename the cols as entity_1 and entity_2
# # prepare the final X_train using pandas transforms

In [29]:
"""
    Performing over sampling is compute expensive
    for local runs. Hence Right now we shall
    select and balance the dataset manual for trial runs
    This introduces a selection bias in the dataset
    But nevertheless, we can perform SMOTE or SMOTE-ENN
    or SMOTE with RandomUnderSampler to bring back the
    dataset size approx to the original.

    This can be done with distributed training in Apache beam
    A sample pipeline for which has been developed along with this
    Submission.

    Caution: The Apache Beam pipeline is highly error prone and requires debugging
"""

# Class based manual balancing of data set introducing selection bias
df_sample = pd.concat(
    [df_clean[(df_clean.tag == 0)][:25000], df_clean[(df_clean.tag == 1)][:25000]],
)

df_sample.tag.value_counts()

0    25000
1    25000
Name: tag, dtype: int64

### Perform train test split 

A more efficient splitting can be employed using tfx pipelines as described in tfx pipeline files

### Preprocessing Model Inputs

Preprocessing model inputs involves tokenizing,
encoding and padding the inputs as shown below

In [ ]:
# split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_sample[["entity_1", "entity_2"]], df_sample.tag, test_size=0.33, random_state=42)

labeled_data = X_train


# Process the labeled data
def preprocess_model_inputs(labeled_data):
    # company_pairs = list(zip(labeled_data['entity_1'], labeled_data['entity_2']))
    # labels = np.array(y_train)

    # Tokenize and pad the company names
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(list(labeled_data['entity_1']) + list(labeled_data['entity_2']))

    # encoding company names
    encoded_company1 = tokenizer.texts_to_sequences(list(labeled_data['entity_1']))
    encoded_company2 = tokenizer.texts_to_sequences(list(labeled_data['entity_2']))

    max_length = max(max(len(seq) for seq in encoded_company1), max(len(seq) for seq in encoded_company2))

    # applying padding to gain equal size for all examples
    padded_company1 = tf.keras.preprocessing.sequence.pad_sequences(encoded_company1, maxlen=max_length)
    padded_company2 = tf.keras.preprocessing.sequence.pad_sequences(encoded_company2, maxlen=max_length)

    return padded_company1, padded_company2, tokenizer, max_length

padded_company1, padded_company2, tokenizer, max_length = preprocess_model_inputs(labeled_data)

### Model Building

In [79]:
def create_siamese_model(vocab_size, embedding_dim, lstm_units):
    input_a = Input(shape=(None,))
    input_b = Input(shape=(None,))

    # Embedding layer for mapping tokens to vectors
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

    # Shared LSTM layer
    shared_lstm = LSTM(lstm_units)

    # Process each input through the embedding and LSTM layers
    output_a = shared_lstm(embedding_layer(input_a))
    output_b = shared_lstm(embedding_layer(input_b))

    # Calculate L1 distance between the two representations
    l1_distance = Lambda(lambda x: tf.abs(x[0] - x[1]))([output_a, output_b])

    # Dense layer to make the final prediction
    dense1 = Dense(50, activation='relu')(l1_distance)
    droput = Dropout(0.2)(dense1)
    dense2 = Dense(20, activation='relu')(droput)
    prediction = Dense(1, activation='sigmoid')(dense2)

    # Create the siamese model
    siamese_model = Model(inputs=[input_a, input_b], outputs=prediction)

    return siamese_model


### Training the model

In [80]:
# Create the siamese model
vocab_size = len(tokenizer.word_index) + 1  # Adjust based on your vocabulary size
embedding_dim = 50  # Adjust based on your embedding dimension
lstm_units = 100

siamese_model = create_siamese_model(vocab_size, embedding_dim, lstm_units)

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
siamese_model.fit([padded_company1, padded_company2], y_train.to_numpy(), epochs=1)

1047/1047 [==============================] - 53s 47ms/step - loss: 0.1529 - accuracy: 0.9454


In [98]:
X_test.head()

,entity_1,entity_2
1309879,dolco gmbh,sxp schulz xtruded
148061,hefei rishang electrical appliances coltd,engineered automation systems
1985823,cambridgeport air systems,cambridgeport air systems inc
2345376,american tile company,zhejiang xinghuali fiber coltd
1132154,seyeon inet co,omz pao


### Making predictions

In [99]:
def make_predictions(x_test, threshold=0.5):
    
    x_test_padded1, x_test_padded2, _, _ = preprocess_model_inputs(x_test)
    similarity_score = siamese_model.predict([np.array(x_test_padded1), np.array(x_test_padded2)])

    return [int(x) for x in similarity_score >= threshold]

In [107]:
start_idx = 0
end_idx = 30000
pred_list = []
while end_idx <= X_test.shape[0]:
    y_pred = make_predictions(X_test[start_idx: end_idx])
    pred_list.append(y_pred)
    start_idx += 30000
    end_idx += 30000

938/938 [==============================] - 6s 6ms/step


### Model Evaluation

here we do notice that despite being trained on such a small
data set the model has captured the relation ships quite well
For experimentation purposes the x_test used is quite larger
than the X_train dataset and still the model was able to
reproduce a precision of 97% and a recall of 84% which is 
quite impressive for the initial trials

In [112]:
y_pred = [val for op in pred_list for val in op]
y_pred = np.array(y_pred)


In [114]:
y_test = y_test[:y_pred.shape[0]]
y_test.shape

(1110000,)

In [115]:
m = tf.keras.metrics.Precision()
m.update_state(y_test, y_pred)
m.result().numpy()

0.9767244

In [116]:
m = tf.keras.metrics.Recall()
m.update_state(y_test, y_pred)
m.result().numpy()

0.8436587

In [121]:
siamese_model.save_weights("siamese_LSTM_embedding.h5")

In [123]:
siamese_model.save("siamese_model", save_format="tf")

INFO:tensorflow:Assets written to: siamese_model\assets


INFO:tensorflow:Assets written to: siamese_model\assets


### Some useful code snippets

In [ ]:
# Further tried handy code snippets for faster functioning

# """ Logic to convert csv data to TFRecords for faster processing in the tfx pipeline
# """

# df_sample.to_csv(Path().cwd() / "data" / "ds_challenge_alpas_sampled.csv")

# import csv
# from pathlib import Path
# import tensorflow as tf
# from tqdm import tqdm


# def _bytes_feature(value):
#     return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))


# def _int64_feature(value):
#     return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# def clean_rows(row):
#     if row["tag"]:
#         row["tag"] = int(row["tag"])
#     return row


# # def convert_zipcode_to_int(zipcode):
# #     if isinstance(zipcode, str) and "XX" in zipcode:
# #         zipcode = zipcode.replace("XX", "00")
# #     int_zipcode = int(zipcode)
# #     return int_zipcode


# original_data_file = Path().cwd() / "data" / "ds_challenge_alpas_sampled.csv"
# tfrecords_filename = Path().cwd() / "tf_records_small" / "ds_challenge_alpas_sampled.tfrecords"
# tf_record_writer = tf.io.TFRecordWriter(str(tfrecords_filename))

# with open(str(original_data_file), encoding="utf8") as csv_file:
#     reader = csv.DictReader(csv_file, delimiter=",", quotechar='"')
#     for row in tqdm(reader):
#         row = clean_rows(row)
#         example = tf.train.Example(
#             features=tf.train.Features(
#                 feature={
#                     "entity_1": _bytes_feature(row["entity_1"]),
#                     "entity_2": _bytes_feature(row["entity_2"]),
#                     "tag": _int64_feature(row["tag"])
#                 }
#             )
#         )
#         tf_record_writer.write(example.SerializeToString())
#     tf_record_writer.close()
